### Regex ile Değişken Türetmek

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [2]:
def load():
    data = pd.read_csv("titanic.csv")
    return data
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


* DİKKAT: Şimdi buradaki titlelar (Miss, Mrs gibi) bizim için önemli olabilir. Bunları konu anlatım kısmında değerlendirmiştik. Şimdi ki amacımız bunları çekmektir. Yapıyı inceleyip bir örüntü üretmeye çalışıyoruz. Baktığımızda Mr, Mrs ve Miss gibi yapıların öncesinde bir boşluk sonrasında da nokta var, büyük küçük harfleri görüyoruz. Anlamalıyız ki regeximizi küçük ya da büyük harfleri seçmesi açısından ve benzer şekilde boşlukla başlayıp nokta ile biteceğini ifade edecek şekilde oluşturmalıyız. 

In [3]:
df['NEW_TITLE'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_TITLE
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,Mr


* DİKKAT: Harika bir değişken geldi. NEW_TITLE adındaki bu değişkenin harika olduğunu nasıl biliyoruz hemen gözlemleyelim.

In [4]:
# new_title, survived, age'i seç...
# new_title'a göre groupby'a al...
# yaş değişkenine göre survived değişkeninin ortalamasını al...
# yaş değişkeninin de ortalamasını ve count'unu al...
df[["NEW_TITLE", "Survived", "Age"]].groupby(["NEW_TITLE"]).agg({"Survived": "mean", "Age": ["count", "mean"]})

Survived   Age       
              mean count   mean
NEW_TITLE                      
Capt         0.000     1 70.000
Col          0.500     2 58.000
Countess     1.000     1 33.000
Don          0.000     1 40.000
Dr           0.429     6 42.000
Jonkheer     0.000     1 38.000
Lady         1.000     1 48.000
Major        0.500     2 48.500
Master       0.575    36  4.574
Miss         0.698   146 21.774
Mlle         1.000     2 24.000
Mme          1.000     1 24.000
Mr           0.157   398 32.368
Mrs          0.792   108 35.898
Ms           1.000     1 28.000
Rev          0.000     6 43.167
Sir          1.000     1 49.000

* DİKKAT: Öyle değerli bir bilgi elde ettik ki gözlenme ihtimali olmayan bir değişken yani bizim için dikkate alınma ihtimali olmayan bir değişkenden dahi çok değerli bir bilgi çıkardık. Nedir o bir bakalım:
* Kaptan varmış,Col vb isimler varmış. Daha öncesinde ifade ettiğimiz gibi hepsine yaş değişkeninin ortalamasını atamıştık, artık master sınıfı için 36 tane varmış bundan. Yaş değişkeni ile ilgili ortalama atama işlemi yapmak istersek bunu artık böyle yapmamamız gerektiğini biliyoruz. Mater'ın yaş ortalaması 4.57. Diğer sınıflara da göz attığımızda aynı durumları görüyoruz. Bunlar kayda değerdir. Şu anda veri setinde daha önce olmayan bir değişken türettik, bu değişken üzerinden yaşların ortalamasını aldığımızda gördük ki hani kategorik değişken kırılımında eksiklikleri atayalım demiştik ya, buna hizmet edebilecek bir değişken türedi. Şimdi bu kategorik değişken kırılımında ortalamaları doldursak daha mantıklı olur gibi gözüküyor. Frekansı da göz önünde bulundurarak yorumlarımızı yapmayı unutmayalım.